In [41]:
class Node:
    def __init__(self,type,id):
        self.type=type
        self.id=id
nodes={}

In [42]:
head={}
edges={}
tot=0
class Edge:
    def __init__(self,path=[]):
        self.path=path
        self.start=path[0]
        self.end=path[-1]
        self.len=len(path)-1
        
def Connect(path):
    edge=Edge(path)
    if(path[0] not in edges):
        edges[path[0]]=[]
    edges[path[0]].append(edge)

In [43]:
import json

problem=json.load(open("T1.json"))
# problem

In [44]:
weather=problem["weather"]
weight=problem["weight"]
budget=problem["budget"]
income=problem["income"]
ddl=problem["ddl"]

water_price=problem["water_price"]
water_per_weight=problem["water_per_weight"]
water_comsumption=problem["water_comsumption"]
food_price=problem["food_price"]
food_per_weight=problem["food_per_weight"]
food_comsumption=problem["food_comsumption"]

weather+=[0,0,0,0,0,0]

In [45]:
for edge in problem["path"]:
    Connect(edge)
    
for node in problem["node"]:
    nodes[node["id"]]=Node(node["type"],node["id"])
    if(node["type"]=="mine"):
        Connect([node["id"],node["id"]])

In [61]:
def ShowTrack(tabel,f=None):
    for i in range(len(tabel["node"])):
        print("Date:{} Node:{} water:{} food:{} money:{} weight:{}".format(
            i,tabel["node"][i],
            tabel["water"][i],
            tabel["food"][i],
            tabel["money"][i],
            tabel["water"][i]*water_per_weight+tabel["food"][i]*food_per_weight
            ))
        if f!=None:
            f.write("{}\t{}\t{}\t{}\t{}".format(
                i,
                tabel["node"][i],
                tabel["money"][i],
                tabel["water"][i],
                tabel["food"][i]
                )+"\n")

In [47]:
def Track(date,water_buy0,show1=False,show2=False):
    tabel={
        "node":[track[i] for i in range(date+1)],
        "water":[0 for i in range(date+1)],
        "food":[0 for i in range(date+1)],
        "money":[incomes[i] for i in range(date+1)],
    }
    tabel["money"][0]=budget
    for i in range(1,date+1):
        for ele in water_deal[i]:
            tabel["water"][i]-=ele[1]
            if(ele[0]==0):
                tabel["water"][0]+=ele[1]
                tabel["money"][0]-=ele[1]*water_price
            else:
                tabel["water"][ele[0]]+=ele[1]
                tabel["money"][ele[0]]-=ele[1]*water_price*2
                    
        for ele in food_deal[i]:
            tabel["food"][i]-=ele[1]
            if(ele[0]==0):
                tabel["food"][0]+=ele[1]
                tabel["money"][0]-=ele[1]*food_price
            else:
                # print((i,ele))
                tabel["food"][ele[0]]+=ele[1]
                tabel["money"][ele[0]]-=ele[1]*food_price*2
    if show1:
        ShowTrack(tabel)
    water,food,max_weight=0,0,[]
    for i in range(date+1):
        water+=tabel["water"][i]
        food+=tabel["food"][i]
        weight_=water*water_per_weight+food*food_per_weight
        max_weight.append(max(weight_,max_weight[i-1] if i>0 else 0))
        capacity=weight
        for ele in water_deal[i]:
            if(ele[0]==0 or ele[1]==0):
                continue
            influence_date=ele[0]-1
            capacity=min(floor((weight-max_weight[influence_date])/water_per_weight),water_buy0)
            if(ele[1]>=capacity):
                tabel["water"][0]+=capacity
                tabel["money"][0]-=capacity*water_price
                tabel["water"][ele[0]]-=capacity
                tabel["money"][ele[0]]+=capacity*water_price*2
                capacity=0
                break
            else:
                tabel["water"][0]+=ele[1]
                tabel["money"][0]-=ele[1]*water_price
                tabel["water"][ele[0]]-=ele[1]
                tabel["money"][ele[0]]+=ele[1]*water_price*2
                capacity-=ele[1]
                max_weight[influence_date]+=ele[1]*water_per_weight
                max_weight[i]=max(max_weight[i],max_weight[influence_date])
                
        if capacity==0:
            break
            
    if show2:
        ShowTrack(tabel)
        
    for i in range(1,date+1):
        tabel["money"][i]+=tabel["money"][i-1]
        tabel["water"][i]+=tabel["water"][i-1]
        tabel["food"][i]+=tabel["food"][i-1]
        if(tabel["water"][i]*water_per_weight+tabel["food"][i]*food_per_weight>weight):
            tabel["money"][i]=-114514
        
    return tabel

In [48]:
from math import floor
water_deal={i:[] for i in range(ddl+3)}
food_deal={i:[] for i in range(ddl+3)}
track=[0 for i in range(ddl+3)]
incomes=[0 for i in range(ddl+3)]

def Move(node,date,w0,w1,b0,b1,uw,m,k,last_load=0):
        track[date+1]=node
        # print(node,end=" ")
        w0=min(w0,w1)
        
        food_com=food_comsumption[weather[date+1]]*k #由于食物价格更高,优先在起点购买更多食物
        food_buy0=min(food_com,floor(w0/food_per_weight))
        food_buy0=min(food_buy0,floor(b0/food_price))
        food_buy1=food_com-food_buy0
        
        w0-=food_buy0*food_per_weight
        w1-=food_buy0*food_per_weight+food_buy1*food_per_weight
        b0-=food_buy0*food_price
        b1-=food_buy0*food_price+food_buy1*food_price*2
        m-=food_buy0*food_price+food_buy1*food_price*2
        food_deal[date+1]=[[0,food_buy0],[last_load,food_buy1]]
        
        water_com=water_comsumption[weather[date+1]]*k
        if last_load!=0: #由于水价格更低,优先在村庄购买更多水
            water_buy1=min(water_com,floor(b1/water_price)-water_com)#(S-x)*p+x*2p<=B ===> x<=B/p-S
            water_buy0=water_com-water_buy1
        else:
            water_buy0=water_com
            water_buy1=0
            
        uw+=water_buy1
        
        w0-=water_buy0*water_per_weight
        w1-=water_buy0*water_per_weight+water_buy1*water_per_weight
        b0-=water_buy0*water_price
        b1-=water_buy0*water_price+water_buy1*water_price*2
        m-=water_buy0*water_price+water_buy1*water_price*2
        water_deal[date+1]=[[0,water_buy0],[last_load,water_buy1]]
        
        incomes[date+1]=income if k==3 else 0
        return w0,w1,b0,b1,uw,m

def Forward(edge,date,left_weight0,left_weight1,left_budget0,left_budget1,used_water_1,money,last_load=0):
    d,w0,w1,b0,b1,uw,m=date,left_weight0,left_weight1,left_budget0,left_budget1,used_water_1,money
    # print(d)
    edge=Edge(edge)
    track[date]=edge.start
    for i in range(edge.len):#枚举路径上的每一步
        if(d>ddl):
            break
        
        if edge.path[i+1]<0:#在矿场休息
            w0,w1,b0,b1,uw,m=Move(-edge.path[i+1],d,w0,w1,b0,b1,uw,m,1,last_load)
            d+=1
            continue
        
        if abs(edge.path[i])==edge.path[i+1]:#挖矿
            w0,w1,b0,b1,uw,m=Move(edge.path[i+1],d,w0,w1,b0,b1,uw,m,3,last_load)
            d+=1
            m+=income
            continue
        
        
        while(edge.path[i]!=edge.path[i+1] and d+1<=ddl and weather[d+1]==2):#沙尘暴被迫停留
            w0,w1,b0,b1,uw,m=Move(edge.path[i],d,w0,w1,b0,b1,uw,m,1,last_load)
            d+=1
            if(edge.path[i] in nodes and nodes[edge.path[i]].type=="village"):#村庄补充物资
                w1=weight
                b1=m
                last_load=d
            # print((d,w0,w1,b0,b1,uw,m))
            if(w0<0 or w1<0 or b0<0 or b1<0):
                d=ddl+1
            # print("Sand Storm")
            # print(d)
        w0,w1,b0,b1,uw,m=Move(edge.path[i+1],d,w0,w1,b0,b1,uw,m,2,last_load)#前进
        d+=1
        if(edge.path[i] in nodes and nodes[edge.path[i]].type=="village"):#村庄补充物资
            w1=weight
            b1=m
            last_load=d
        # print((d,w0,w1,b0,b1,uw,m))
        if(w0<0 or w1<0 or b0<0 or b1<0):
            d=ddl+1
        # print(d)
    return d,w0,w1,b0,b1,uw,m,last_load

In [49]:
ans=0
tabel={}
def Dfs(end,path):
    if len(path)>ddl:
        # print(path)
        return 0
    d,w0,w1,b0,b1,uw,m,lst=0,weight,weight,budget,budget,0,budget,0
    d,w0,w1,b0,b1,uw,m,lst=Forward(path,d,w0,w1,b0,b1,uw,m,lst)
    if(d>ddl):
        return 0
    # if path==[1,25,24,23,22,9,15,14,12,12,12,12,12,12,12]:
    #     print("fuck")
    #     print(d)
    if path[-1]==end:
        # print(path)
        global tabel
        global ans
        tabel_=Track(d,uw)
        money=tabel_["money"][d]
        if(money>ans):
            ans=money
            tabel=tabel_
            print((ans,d,uw))
        return money
        
    profit=0
    for edge in edges[abs(path[-1])]:
        if(edge.len==1 and edge.end==edge.start and weather[d+1]==2):
            profit=max(profit,Dfs(end,path+[-abs(path[-1])]))
        profit=max(profit,Dfs(end,path+edge.path[1:]))
        
    return profit
            
    

In [50]:
for node in nodes:
    if(nodes[node].type=="start"):
        st=node
    if(nodes[node].type=="end"):
        ed=node

In [51]:
ans=Dfs(end=ed,path=[st])
print(ans)

(9410, 3, 0)
(9670, 30, 343)
(10070, 30, 363)
(10575, 24, 263)
10575


In [62]:
with open("result.txt","w") as f:
    ShowTrack(tabel=tabel,f=f)

Date:0 Node:1 water:167 food:349 money:5675 weight:1199
Date:1 Node:25 water:151 food:337 money:5675 weight:1127
Date:2 Node:24 water:135 food:325 money:5675 weight:1055
Date:3 Node:23 water:125 food:311 money:5675 weight:997
Date:4 Node:23 water:115 food:301 money:5675 weight:947
Date:5 Node:22 water:105 food:287 money:5675 weight:889
Date:6 Node:9 water:89 food:275 money:5675 weight:817
Date:7 Node:9 water:79 food:265 money:5675 weight:767
Date:8 Node:15 water:69 food:251 money:5675 weight:709
Date:9 Node:14 water:237 food:239 money:3835 weight:1189
Date:10 Node:12 water:221 food:227 money:3835 weight:1117
Date:11 Node:12 water:211 food:217 money:3835 weight:1067
Date:12 Node:12 water:187 food:199 money:4835 weight:959
Date:13 Node:12 water:172 food:178 money:5835 weight:872
Date:14 Node:12 water:148 food:160 money:6835 weight:764
Date:15 Node:12 water:124 food:142 money:7835 weight:656
Date:16 Node:12 water:100 food:124 money:8835 weight:548
Date:17 Node:12 water:90 food:114 money:8

In [55]:
d,w0,w1,b0,b1,uw,m,lst=0,weight,weight,budget,budget,0,budget,0
d,w0,w1,b0,b1,uw,m,lst=Forward([1,25,24,23,22,9,15],d,w0,w1,b0,b1,uw,m,lst)
d,w0,w1,b0,b1,uw,m,lst

(8, 710, 710, 8530, 8530, 0, 8530, 0)

In [56]:
d,w0,w1,b0,b1,uw,m,lst=Forward([15,14,12],d,w0,w1,b0,b1,uw,m,lst)
d,w0,w1,b0,b1,uw,m,lst

(10, 614, 1128, 8210, 8050, 16, 8050, 9)

In [57]:
for i in range(6):
    w0,w1,b0,b1,uw,m=Move(12,d,w0,w1,b0,b1,uw,m,3,lst)
    d+=1
    m+=income
for i in range(2):
    w0,w1,b0,b1,uw,m=Move(12,d,w0,w1,b0,b1,uw,m,1,lst)
    d+=1
for i in range(1):
    w0,w1,b0,b1,uw,m=Move(12,d,w0,w1,b0,b1,uw,m,3,lst)
    d+=1
    m+=income
d,w0,w1,b0,b1,uw,m,lst

(19, 292, 251, 6600, 4590, 201, 11590, 9)

In [58]:
d,w0,w1,b0,b1,uw,m,lst=Forward([12,14,15],d,w0,w1,b0,b1,uw,m,lst)
d,w0,w1,b0,b1,uw,m,lst

(21, 151, 121, 6340, 4070, 227, 11070, 9)

In [59]:
d,w0,w1,b0,b1,uw,m,lst=Forward([15,9,21,27],d,w0,w1,b0,b1,uw,m,lst)
d,w0,w1,b0,b1,uw,m,lst

(24, 41, 1070, 5940, 10310, 263, 10310, 22)

In [60]:
tabel_=Track(d,uw)
ShowTrack(tabel_)

Date:0 Node:1 water:167 food:349 money:5675 weight:1199
Date:1 Node:25 water:151 food:337 money:5675 weight:1127
Date:2 Node:24 water:135 food:325 money:5675 weight:1055
Date:3 Node:23 water:125 food:311 money:5675 weight:997
Date:4 Node:23 water:115 food:301 money:5675 weight:947
Date:5 Node:22 water:105 food:287 money:5675 weight:889
Date:6 Node:9 water:89 food:275 money:5675 weight:817
Date:7 Node:9 water:79 food:265 money:5675 weight:767
Date:8 Node:15 water:69 food:251 money:5675 weight:709
Date:9 Node:14 water:237 food:239 money:3835 weight:1189
Date:10 Node:12 water:221 food:227 money:3835 weight:1117
Date:11 Node:12 water:191 food:197 money:4835 weight:967
Date:12 Node:12 water:167 food:179 money:5835 weight:859
Date:13 Node:12 water:152 food:158 money:6835 weight:772
Date:14 Node:12 water:128 food:140 money:7835 weight:664
Date:15 Node:12 water:104 food:122 money:8835 weight:556
Date:16 Node:12 water:80 food:104 money:9835 weight:448
Date:17 Node:12 water:70 food:94 money:9835

In [ ]:
pth=[1,25,24,23,22,9,15,14,12,12,12,12,12,12,12,-12,-12,12,14,15,9,21,27]
print(len(pth))
d,w0,w1,b0,b1,uw,m,lst=0,weight,weight,budget,budget,0,budget,0
d,w0,w1,b0,b1,uw,m,lst=Forward(pth,d,w0,w1,b0,b1,uw,m,lst)
d,w0,w1,b0,b1,uw,m,lst

23


(24, 41, 1070, 5940, 10310, 263, 10310, 22)

In [ ]:
tabel_=Track(d,uw)
ShowTrack(tabel_)

Date:0 Node:1 water:167 food:349 money:5675 weight:1199
Date:1 Node:25 water:151 food:337 money:5675 weight:1127
Date:2 Node:24 water:135 food:325 money:5675 weight:1055
Date:3 Node:23 water:125 food:311 money:5675 weight:997
Date:4 Node:23 water:115 food:301 money:5675 weight:947
Date:5 Node:22 water:105 food:287 money:5675 weight:889
Date:6 Node:9 water:89 food:275 money:5675 weight:817
Date:7 Node:9 water:79 food:265 money:5675 weight:767
Date:8 Node:15 water:69 food:251 money:5675 weight:709
Date:9 Node:14 water:237 food:239 money:3835 weight:1189
Date:10 Node:12 water:221 food:227 money:3835 weight:1117
Date:11 Node:12 water:191 food:197 money:4835 weight:967
Date:12 Node:12 water:167 food:179 money:5835 weight:859
Date:13 Node:12 water:152 food:158 money:6835 weight:772
Date:14 Node:12 water:128 food:140 money:7835 weight:664
Date:15 Node:12 water:104 food:122 money:8835 weight:556
Date:16 Node:12 water:80 food:104 money:9835 weight:448
Date:17 Node:12 water:70 food:94 money:9835